In [1]:
var('p,t')
import numpy as np
extra(p,t,r)=(p^(r+1)-p^r)/(p^(r+1)-1)
#Check if bArr[i]/(eArr[i]/eBase) is an integer for all i
def checkInt(bArr,eArr,eBase):
    for i in range(len(bArr)):
        if(not(bArr[i]/(eArr[i]/eBase) in ZZ)):
            return False
    return True
#it's the inc defined in the paper
def inc(bArr,eArr,eBase):
    beta = min([bArr[i]/(eArr[i]/eBase) for i in range(len(bArr))])
    bArrCopy = deepcopy(bArr)
    for i in range(len(bArrCopy)):
        if(bArrCopy[i]/(eArr[i]/eBase) == beta):
            bArrCopy[i] += 1
    return bArrCopy
#returns a list of dictionaries, each dictionary is a function from a partition, to possible values of nO on the partition
#The point is that the valid partitions are just partitions of Is, except you add in notIs to one of the sets in the partition
#(or is a set itself in the partition). So, it's like having a partition of Is union {-1}, and then replace -1 with notIs
def validPartsnO(eArr,fArr,eBase,fBase,bArr):
    beta = min([bArr[i]/(eArr[i]/eBase) for i in range(len(bArr))])
    dbeta = beta.denominator()
    Is = []
    notIs = []
    for i in range(len(bArr)):
        if(bArr[i]/(eArr[i]/eBase) == beta):
            Is.append(i)
        else:
            notIs.append(i)
    Is = set(Is)
    notIs = set(notIs)
#     print(Is)
    if(len(notIs) != 0):
        Is.add(-1)
    parts = SetPartitions(Is)
    validPartsnODict = []
    for partset in parts:
        validnODict = {}
        newpartset = []
        for partElt in partset:
            if(-1 in partElt):
                newpart = set(partElt)
                newpart.remove(-1)
                newpart = newpart.union(notIs)
                newpart=frozenset(newpart)   

                validnODict[newpart] = [1]
                newpartset.append(newpart)
            else:
                newpart = partElt
                validnODict[newpart] = [dbeta*j for j in divisors(gcd([fArr[i]/fBase for i in partElt]))]
                if(dbeta != 1):
                    validnODict[newpart].append(1)
                newpartset.append(newpart)

        if(len(newpart)==len(bArr) and (1 in validnODict[newpart])):
            validnODict[newpart].remove(1)
        validPartsnODict.append(validnODict)
            
    return validPartsnODict
def arrSubset(arr,indices):
    newArr = []
    for i in indices:
        newArr.append(arr[i])
    return newArr
def h(beta,nO):
    if(nO == 1):
        return 1
    else:
        return denominator(beta)
def shortP(fBase, nOprime):
    flist = factor(nOprime)
    primeFactorSubsets = Subsets([flist[i][0] for i in range(0,len(flist))])
    tot = 0
    for subset in primeFactorSubsets:
        prod = 1
        for q in subset:
            prod *= q
        tot += (-1)^(len(subset))*(p^(fBase*nOprime/prod)-1)
    return tot/nOprime
def P(eArr,fArr,eBase,fBase,bArr,part,nOArr):
    beta = min([bArr[i]/(eArr[i]/eBase) for i in range(len(bArr))])
    dbeta = denominator(beta)
    nODictCount = {}
    fieldnOCount = {}
    Is = []
    notIs = []
    for i in range(len(bArr)):
        if(bArr[i]/(eArr[i]/eBase) == beta):
            Is.append(i)
        else:
            notIs.append(i)
    
    for i in range(len(part)):
        if(nOArr[i] in fieldnOCount):
            fieldnOCount[nOArr[i]] += len(part[i])
        else:
            fieldnOCount[nOArr[i]] = len(part[i])
    
    for i in range(len(nOArr)):
        if(nOArr[i] in nODictCount):
            nODictCount[nOArr[i]] += 1
        else:
            nODictCount[nOArr[i]] = 1
    
    if((not (dbeta == 1)) and 1 in nODictCount.keys() and nODictCount[1] > 1):
        return 0
    
    prod = 1
    for k in nODictCount.keys():
        if(k != 1):
            Nk = (k/dbeta)^(fieldnOCount[k])*falling_factorial(shortP(fBase,k/dbeta), nODictCount[k])
            #falling_factorial(x,a)=x(x-1)...(x-a+1)=x!/(x-a)!.
        else: 
            if(dbeta != 1):
                Nk = 1
            else:
                if(len(Is) == len(eArr)):
                    Nk = falling_factorial(p^(fBase), nODictCount[k])
                else:
                    Nk = falling_factorial(p^(fBase)-1, nODictCount[k]-1)
        prod *= Nk
    return prod
def G(eArr,fArr,eBase,fBase,bArr):
    if(len(eArr) == 1 and eArr[0] == eBase and fArr[0] == fBase):
        Gtot(p,t)=p^(-fBase*bArr[0])
        return Gtot
    kLim = max([ceil(bArr[i]/(eArr[i]/eBase)) for i in range(len(bArr))])
#     print(kLim)
    Gtot = 0
#     print("hi",eArr,fArr,eBase,fBase,bArr)
    while(not checkInt(bArr,eArr,eBase)):
        Gtot += H(eArr,fArr,eBase,fBase,bArr)
#         print((H(eArr,fArr,eBase,fBase,bArr)(p,1)).simplify_full(),eArr,fArr,eBase,fBase,bArr)
        bArr = inc(bArr,eArr,eBase)
    d = sum([eArr[i]*fArr[i]/(eBase*fBase) for i in range(len(eArr))])
#     print((Gtot(p,t=1)).simplify_full(),eArr,fArr,eBase,fBase,bArr)
# #     print("end")
    G0Builder = 0
    bArr = [0 for i in range(len(bArr))]
    G0Builder += H(eArr,fArr,eBase,fBase,bArr)
    bArr = inc(bArr,eArr,eBase)
#     print("hi",G0Builder,eArr,fArr,eBase,fBase,bArr)
    while(not checkInt(bArr,eArr,eBase)):
        G0Builder += p^(fBase)*H(eArr,fArr,eBase,fBase,bArr)
#         print(G0Builder,eArr,fArr,eBase,fBase,bArr)
        bArr = inc(bArr,eArr,eBase)
    G0Builder *= (1-p^(fBase*(1-d))*t^(d*(d-1)/eBase))^(-1)
#     print("end",G0Builder,eArr,fArr,eBase,fBase,bArr)

    G0 = (p^(-fBase*d)*t^(d*(d-1)/eBase))^kLim*G0Builder
#     print((1-p^(fBase*(1-d))*t^(d*(d-1)/eBase))^(-1))
#     print(G0(p,1).simplify_full(),eArr,fArr,eBase,fBase,bArr)
#     print((Gtot+G0)(p,1).simplify_full(),eArr,fArr,eBase,fBase,bArr)
    return Gtot + G0
def fac(f):
    return (f(p=p,t=t)/f(p=p^(-1),t=t^(-1))).canonicalize_radical().simplify_full()
def H(eArr,fArr,eBase,fBase,bArr):
    beta = min([bArr[i]/(eArr[i]/eBase) for i in range(len(bArr))])
    d = sum([eArr[i] * fArr[i]/(eBase * fBase) for i in range(0,len(eArr))])
    Gtot = 0
#     print(validPartsnO(eArr,fArr,eBase,fBase,bArr),eArr,fArr,eBase,fBase,bArr)
#     print("start",eArr,fArr,eBase,fBase,bArr)
    for validnODict in validPartsnO(eArr,fArr,eBase,fBase,bArr):
        for nOArr in cartesian_product(validnODict.values()).list():
            klist = list(validnODict.keys())
            Gprod = 1
            bArrCopy = inc(bArr,eArr,eBase)
            for i in range(len(klist)):
#                 print(G(arrSubset(eArr,klist[i]),arrSubset(fArr,klist[i]),eBase*h(beta,nOArr[i]),fBase*nOArr[i]/h(beta,nOArr[i]),arrSubset(bArrCopy,klist[i])))
#                 print(arrSubset(eArr,klist[i]),arrSubset(fArr,klist[i]),eBase*h(beta,nOArr[i]),fBase*nOArr[i]/h(beta,nOArr[i]),arrSubset(bArrCopy,klist[i]))
                Gprod *= G(arrSubset(eArr,klist[i]),arrSubset(fArr,klist[i]),eBase*h(beta,nOArr[i]),fBase*nOArr[i]/h(beta,nOArr[i]),arrSubset(bArrCopy,klist[i]))(p,t^(nOArr[i]))
            texponent = eBase^(-1)*beta*(d*(d-1)-sum([sum([(eArr[i]*fArr[i])/(eBase*fBase) for i in klist[l]]) * (sum([(eArr[i]*fArr[i])/(eBase*fBase)/nOArr[l] for i in klist[l]])-1) for l in range(len(klist))]))
#             print(eBase)
#             print(beta)
#             print(d)
#             print(sum([sum([(eArr[i]*fArr[i])/(eBase*fBase) * (sum([(eArr[i]*fArr[i])/(eBase*fBase)/nOArr[l] for j in klist[i]])-1) for i in klist[l]]) for l in range(len(klist))]))
#             print(texponent,eArr,fArr,eBase,fBase,bArr,nOArr)
#             print(fac(P(eArr,fArr,eBase,fBase,bArr,klist,nOArr)*t^texponent*Gprod),P(eArr,fArr,eBase,fBase,bArr,klist,nOArr)*t^texponent*Gprod,eArr,fArr,eBase,fBase,bArr)
            Gtot += P(eArr,fArr,eBase,fBase,bArr,klist,nOArr)*t^texponent*Gprod
#             print(klist,nOArr,P(eArr,fArr,eBase,fBase,bArr,klist,nOArr))
#             print(Gprod, arrSubset(eArr,klist[i]),arrSubset(fArr,klist[i]),eBase*h(beta,nOArr[i]),fBase*nOArr[i]/h(beta,nOArr[i]),arrSubset(bArrCopy,klist[i]))
#             print(P(eArr,fArr,eBase,fBase,bArr,klist,nOArr),eArr,fArr,eBase,fBase,bArr,klist,nOArr)
#     if(eBase == fBase == 1):
#         print(Gtot)
#     print("end",eArr,fArr,eBase,fBase,bArr)
#     print("done")
    return Gtot
def w(fBase, d):
    q = p^fBase
    return (q^(d+1)-q^d)/(q^(d+1)-1)
def perm(eArr,fArr,eBase,fBase):
    m = len(eArr)
    eRel = [eArr[i]/eBase for i in range(m)]
    fRel = [fArr[i]/fBase for i in range(m)]
    
    hashTable = {}
    for i in range(m):
        if (eRel[i],fRel[i]) in hashTable:
            hashTable[(eRel[i],fRel[i])] += 1;
        else:
            hashTable[(eRel[i],fRel[i])] = 1;
    prod = 1;
    for i in hashTable.values():
        prod *= factorial(i)
    return prod;
# Returns the generating function whose coefficient of t^i is a rational function in terms of q, representing the
# density of polynomials with discriminant i generating an extension with the splitting type specified by eArr and fArr.
# Note that we use absolute ramification index and inertia degree here.
def rhoGF(eArr,fArr,eBase,fBase):
    
    S = subsets(range(len(eArr)))
    total = 0;
    m = len(eArr)
    d = sum(eArr[i] * fArr[i]/(eBase*fBase) for i in range(m))
    q = p^(fBase)
    fProd = 1;
    for i in fArr:
        fProd *= i/fBase
    for s in S:
        S1eArr = [eArr[i] for i in s]
        S1fArr = [fArr[i] for i in s]
        S2eArr = [eArr[i] for i in range(m) if not i in s]
        S2fArr = [fArr[i] for i in range(m) if not i in s]
        
        if(len(s) == 0):
            total += G(S2eArr,S2fArr,eBase,fBase,[1 for i in range(m-len(s))])*p^(sum((S2eArr[i]/eBase - 1)*S2fArr[i] for i in range(m-len(s)))/2)
        if(len(s) == m):
            total += G(S1eArr,S1fArr,eBase,fBase,[0 for i in range(len(s))])*p^(sum((S1eArr[i]/eBase - 1)*S1fArr[i] for i in range(len(s)))/2)
        if(len(s) != 0 and len(s) != m):
            total += G(S1eArr,S1fArr,eBase,fBase,[0 for i in range(len(s))])*G(S2eArr,S2fArr,eBase,fBase,[1 for i in range(m-len(s))])*p^(sum((S2eArr[i]/eBase - 1)*S2fArr[i] for i in range(m-len(s)))/2)*p^(sum((S1eArr[i]/eBase - 1)*S1fArr[i] for i in range(len(s)))/2)
    return w(fBase,d) * total / (perm(eArr,fArr,eBase,fBase)*p^(sum((eArr[i]/eBase-1)*fArr[i] for i in range(m)))*fProd)
def Gtotef11(e,f):
    d = e*f
    return (((G([e],[f],1,1,[0])+G([e],[f],1,1,[1]))(p,t)*extra(p,t,d))*(p^(-(e-1)*f/2))).simplify_full().canonicalize_radical()/f

In [2]:
# As an example, we may find the generating function G((8^6,12^3),3,4,(0,0),p) by using the following command, where we note
# here that everything is in terms of absolute ramification index and inertia degree
G([6,3],[8,12],3,4,[0,0])

/opt/sagemath-9.2/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:134: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
/opt/sagemath-9.2/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:142: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
/opt/sagemath-9.2/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:146: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)


(p^4*((p^12 - p^4)*(p^8 - p^4)*t^8/(p^40*(t^2/p^8 - 1)) - (p^12 - p^4)*(p^8 - p^4)*(p^4 - 1)*(t^2)^(2/3)*t^(22/3)/(p^48*((t^2)^(1/3)/p^4 - 1)*(t^2/p^8 - 1))) + (p^12 - p^4)*((p^4*((p^8 - p^4)*t^2/p^16 - (p^8 - p^4)*(p^4 - 1)*(t^2)^(2/3)*t^(4/3)/(p^24*((t^2)^(1/3)/p^4 - 1))) + (p^8 - p^4)*((p^8 - 1)*(t^2)^(1/3)/p^16 - (p^8 - 1)*t^2/(p^24*((t^2)^(2/3)/p^8 - 1))))*t^4/(p^16*(t^4/p^12 - 1)) - (p^8 - p^4)*t^2/p^16 + (p^8 - p^4)*(p^4 - 1)*(t^2)^(2/3)*t^(4/3)/(p^24*((t^2)^(1/3)/p^4 - 1)))/p^8 - (p^12 - p^4)*(p^8 - p^4)*((p^8 - 1)*(t^2)^(1/3)/p^16 - (p^8 - 1)*t^2/(p^24*((t^2)^(2/3)/p^8 - 1)))/p^12 - (p^12 - p^4)*(p^4 - 1)*t^2*((p^4*((p^8 - p^4)*t^2/p^16 - (p^8 - p^4)*(p^4 - 1)*(t^2)^(2/3)*t^(4/3)/(p^24*((t^2)^(1/3)/p^4 - 1))) + (p^8 - p^4)*((p^8 - 1)*(t^2)^(1/3)/p^16 - (p^8 - 1)*t^2/(p^24*((t^2)^(2/3)/p^8 - 1))))*t^4/(p^16*(t^4/p^12 - 1)) - (p^8 - p^4)*t^2/p^16 + (p^8 - p^4)*(p^4 - 1)*(t^2)^(2/3)*t^(4/3)/(p^24*((t^2)^(1/3)/p^4 - 1)))/(p^20*(t^2/p^8 - 1)) + (p^12 - p^4)*(p^8 - p^4)*t^2*((p^8 - 

In [3]:
# To find the density of polynomials over a local field K with absolute ramification index 3 and inertia degree 4
# generating a tame extension with splitting type (8^6,12^3), we may use the rhoGF, and specify t=1
rhoGF([6,3],[8,12],3,4)(t=1)

/opt/sagemath-9.2/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:134: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
/opt/sagemath-9.2/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:142: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
/opt/sagemath-9.2/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:146: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)


1/6*(p^32 - p^28)*(p^4*((p^4*((p^12 - p^4)*(p^8 - p^4)/(p^40*(1/p^8 - 1)) - (p^12 - p^4)*(p^8 - p^4)*(p^4 - 1)/(p^48*(1/p^4 - 1)*(1/p^8 - 1))) - (p^12 - p^4)*((p^8 - p^4)/p^16 - (p^4*((p^8 - p^4)/p^16 - (p^8 - p^4)*(p^4 - 1)/(p^24*(1/p^4 - 1))) + (p^8 - p^4)*((p^8 - 1)/p^16 - (p^8 - 1)/(p^24*(1/p^8 - 1))))/(p^16*(1/p^12 - 1)) - (p^8 - p^4)*(p^4 - 1)/(p^24*(1/p^4 - 1)))/p^8 - (p^12 - p^4)*(p^8 - p^4)*((p^8 - 1)/p^16 - (p^8 - 1)/(p^24*(1/p^8 - 1)))/p^12 + (p^12 - p^4)*(p^4 - 1)*((p^8 - p^4)/p^16 - (p^4*((p^8 - p^4)/p^16 - (p^8 - p^4)*(p^4 - 1)/(p^24*(1/p^4 - 1))) + (p^8 - p^4)*((p^8 - 1)/p^16 - (p^8 - 1)/(p^24*(1/p^8 - 1))))/(p^16*(1/p^12 - 1)) - (p^8 - p^4)*(p^4 - 1)/(p^24*(1/p^4 - 1)))/(p^20*(1/p^8 - 1)) + (p^12 - p^4)*(p^8 - p^4)*((p^8 - 1)/p^16 - (p^8 - 1)/(p^24*(1/p^8 - 1)))/(p^20*(1/p^8 - 1)))/(p^28*(1/p^24 - 1)) - (p^12 - p^4)*(p^8 - p^4)/(p^40*(1/p^8 - 1)) + (p^12 - p^4)*(p^8 - p^4)*(p^4 - 1)/(p^48*(1/p^4 - 1)*(1/p^8 - 1))) + (p^4*((p^12 - p^4)*(p^8 - p^4)/(p^40*(1/p^8 - 1)) - (p

In [ ]:
# One observes the following functional equation
print((rhoGF([6,3],[8,12],3,4)(p=p,t=t)/rhoGF([6,3],[8,12],3,4)(p=p^(-1),t=t^(-1))).canonicalize_radical().full_simplify())
print((rhoGF([6,2],[4,4],2,2)(p=p,t=t)/rhoGF([6,2],[4,4],2,2)(p=p^(-1),t=t^(-1))).canonicalize_radical().full_simplify())
print((rhoGF([3],[4],1,1)(p=p,t=t)/rhoGF([3],[4],1,1)(p=p^(-1),t=t^(-1))).canonicalize_radical().full_simplify())


/opt/sagemath-9.2/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:134: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
/opt/sagemath-9.2/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:142: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
/opt/sagemath-9.2/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:146: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
